In [ ]:
# default_exp datos

# Datos

> Cargar y preparar datos de rezago social, pobreza, salud, y de infraestructura de México. 

In [ ]:
#hide
from nbdev.showdoc import *

# Bibliotecas

In [ ]:
# export
import os
import glob
import itertools
from fastcore.test import *

import pandas as pd
import geopandas as gpd
from osgeo import gdal, ogr
from datetime import timedelta, date, datetime
import csv

import requests

## Utilería

### columnas_comunes

In [ ]:
#exports
DATA_DIR_COVID = 'datos/secretaria_salud/'


In [ ]:
#exports
def asegura_archivos_covid_disponibles(fechas):
    '''
    Esta función genera un error si no están disponibles todos los archivos de casos COVID-19 en México
    de las fechas especificadas. Los nombres de archivos siguen el patrón con el que los publica la Secretaría de Salud:
    {AÑO}{MES}{DIA}COVID19MEXICO.csv.zip, por ejemplo 200413COVID19MEXICO.csv.zip
    '''
    for fecha in fechas:
        fecha_str = fecha.strftime("%y%m%d")
        file = os.path.join(DATA_DIR_COVID, f'{fecha_str}COVID19MEXICO.csv.zip')
        assert os.path.exists(file), f'{fecha_str} no disponible'


In [ ]:
inicio = pd.to_datetime('200428', yearfirst=True)
fin = pd.to_datetime('200608', yearfirst=True)

asegura_archivos_covid_disponibles(pd.date_range(inicio, fin))

In [ ]:
# exports

def columnas_comunes(df_1, df_2):
    '''
    Obtiene la lista de columnas con el mismo nombre entre dos DataFrame
    '''
    return list(set(df_1.columns).intersection(df_2.columns))

### arregla_cvegeo

In [ ]:
#exports

def arregla_cvegeo(df, campo_cvegeo='cvegeo', extraer_ent=False):
    '''
    Revisa que los indetificadores de municipio (clave entidad + clave municipio) tengan 5 digitos, si un identificador
    tiene 4 le agrega un cero. Esta corrección asume que el problema surgió de leer los identificadores como enteros
    lo cual provoca que se descarten los ceros a la izquierda.
    '''
    df[campo_cvegeo] = df[campo_cvegeo].apply(lambda x: '0' + x if len(x) == 4 else x)
    
    if extraer_ent:
        df[f'ent_{campo_cvegeo}'] = df[campo_cvegeo].str[:2]
    return df

## carga_datos_covid19_MX

In [ ]:
# exports
DATA_DIR_COVID = 'datos/secretaria_salud/'

def carga_datos_covid19_MX(fecha='200601', resolver_claves='si_no_binarias', periodo='dia'):
    """
        Lee en un DataFrame el CSV con el reporte de casos de la Secretaría de Salud de México publicado en una fecha dada. Esta función
        también lee el diccionario de datos que acompaña a estas publicaciones para preparar algunos campos, en particular permite la funcionalidad
        de generar columnas binarias para datos con valores 'SI', 'No'.
        
        **Nota**: En esta versión la ruta esta y nombre de los archivos es fija. Asumimos que existe un directorio 'datos/secretaria_salud/'
        donde se encuentran todos los archivos.
        
        periodo: 'dia', 'historico'. Trae los datos de la fecha indicada o trae
        la serie de conjuntos de datos hasta la fecha indicada (por implementar).
        
        resolver_claves: 'sustitucion', 'agregar', 'si_no_binarias', 'solo_localidades'. Resuelve los valores del conjunto de datos usando el
        diccionario de datos y los catálogos. 'sustitucion' remplaza los valores en las columnas, 'agregar'
        crea nuevas columnas. 'si_no_binarias' cambia valores SI, NO, No Aplica, SE IGNORA, NO ESPECIFICADO por 1, 0, 0, 0, 0 respectivamente.
        
    """
    
    file = os.path.join(DATA_DIR_COVID, f'{fecha}COVID19MEXICO.csv.zip')
    df = pd.read_csv(file, dtype=object, encoding='latin-1')
    
    # Hay un error y el campo OTRA_COMP es OTRAS_COMP según los descriptores
    df.rename(columns={'OTRA_COM': 'OTRAS_COM'}, inplace=True)
    
    # Asignar clave única a municpios
    df['MUNICIPIO_RES'] = df['ENTIDAD_RES'] + df['MUNICIPIO_RES']
    df['CLAVE_MUNICIPIO_RES'] = df['MUNICIPIO_RES']
    
    # Resolver códigos de entidad federal
    entidades = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                              'Catálogo de ENTIDADES',
                              dtype=str, encoding='latin-1')
    
    cols_entidad = ['ENTIDAD_RES', 'ENTIDAD_UM', 'ENTIDAD_NAC']
    df['CLAVE_ENTIDAD_RES'] = df['ENTIDAD_RES']
    df[cols_entidad] = df[cols_entidad].replace(to_replace=entidades['CLAVE_ENTIDAD'].values,
                                               value=entidades['ENTIDAD_FEDERATIVA'].values)
    
    # Leer diccionario y catálogo de datos
    municipios = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                               'Catálogo MUNICIPIOS',
                               dtype=str)
    municipios['CLAVE_MUNICIPIO'] = municipios['CLAVE_ENTIDAD'] + municipios['CLAVE_MUNICIPIO']

    # Resolver códigos de municipio
    df['MUNICIPIO_RES'] = df['MUNICIPIO_RES'].replace(to_replace=municipios['CLAVE_MUNICIPIO'].values,
                                               value=municipios['MUNICIPIO'].values)
        
    # Resolver resultados
    tipo_resultado = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                                   'Catálogo RESULTADO',
                                   dtype=str)
    df['RESULTADO'] = df['RESULTADO'].replace(to_replace=tipo_resultado['CLAVE'].values,
                                                              value=tipo_resultado['DESCRIPCIÓN'].values)
    
    # Resolver datos SI - NO

    # Necesitamos encontrar todos los campos que tienen este tipo de dato y eso 
    # viene en los descriptores, en el campo FORMATO_O_FUENTE
    
    descriptores = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Descriptores_0419.xlsx',
                                 index_col='Nº')
    descriptores.columns = list(map(lambda col: col.replace(' ', '_'), descriptores.columns))
    descriptores['FORMATO_O_FUENTE'] = descriptores.FORMATO_O_FUENTE.str.strip()

    datos_si_no = descriptores.query('FORMATO_O_FUENTE == "CATÁLOGO: SI_ NO"')
    cat_si_no = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                              'Catálogo SI_NO',
                              dtype=str)
    cat_si_no['DESCRIPCIÓN'] = cat_si_no['DESCRIPCIÓN'].str.strip()
    
    campos_si_no = datos_si_no.NOMBRE_DE_VARIABLE
    nuevos_campos_si_no = campos_si_no
    
    if resolver_claves == 'agregar':
        nuevos_campos_si_no = [nombre_var + '_NOM' for nombre_var in campos_si_no]
    elif resolver_claves == 'si_no_binarias':
        nuevos_campos_si_no = [nombre_var + '_BIN' for nombre_var in campos_si_no]
        cat_si_no['DESCRIPCIÓN'] = list(map(lambda val: 1 if val == 'SI' else 0, cat_si_no['DESCRIPCIÓN']))
        
    
    df[nuevos_campos_si_no] = df[datos_si_no.NOMBRE_DE_VARIABLE].replace(
                                                to_replace=cat_si_no['CLAVE'].values,
                                                value=cat_si_no['DESCRIPCIÓN'].values)
    # Resolver tipos de paciente
    cat_tipo_pac = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                                 'Catálogo TIPO_PACIENTE',
                                 dtype=str)
    df['TIPO_PACIENTE'] = df['TIPO_PACIENTE'].replace(to_replace=cat_tipo_pac['CLAVE'].values,
                                                      value=cat_tipo_pac['DESCRIPCIÓN'].values)
    
    df['FECHA_INGRESO'] = pd.to_datetime(df['FECHA_INGRESO'])
    df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])
    df['FECHA_DEF'] = pd.to_datetime(df['FECHA_DEF'], 'coerce')
    df['DEFUNCION'] = (df['FECHA_DEF'].notna()).astype(int)
    df['EDAD'] = df['EDAD'].astype(int)
    
    df.set_index('FECHA_INGRESO', inplace=True)
    df['AÑO_INGRESO'] = df.index.year
    df['MES_INGRESO'] = df.index.month
    df['DIA_SEMANA_INGRESO'] = df.index.weekday
    df['SEMANA_AÑO_INGRESO'] = df.index.weekofyear
    df['DIA_MES_INGRESO'] = df.index.day
    df['DIA_AÑO_INGRESO'] = df.index.dayofyear
        
    return df

In [ ]:
casos_df = carga_datos_covid19_MX(resolver_claves='si_no_binarias')

In [ ]:
casos_df

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO_BIN,MIGRANTE_BIN,UCI_BIN,DEFUNCION,AÑO_INGRESO,MES_INGRESO,DIA_SEMANA_INGRESO,SEMANA_AÑO_INGRESO,DIA_MES_INGRESO,DIA_AÑO_INGRESO
FECHA_INGRESO,,,,,,,,,,,,,,,,,,,,,
2020-05-11,2020-06-01,04f3dd,2,3,SINALOA,2,SINALOA,SINALOA,CULIACÁN,AMBULATORIO,...,1,0,0,0,2020,5,0,20,11,132
2020-05-13,2020-06-01,1607b3,2,3,TABASCO,2,TABASCO,TABASCO,CENTRO,AMBULATORIO,...,1,0,0,0,2020,5,2,20,13,134
2020-05-22,2020-06-01,1b7c4b,2,3,TABASCO,1,TABASCO,TABASCO,COMALCALCO,HOSPITALIZADO,...,0,0,0,0,2020,5,4,21,22,143
2020-04-17,2020-06-01,03f6dd,2,4,CIUDAD DE MÉXICO,1,CIUDAD DE MÉXICO,MÉXICO,NEZAHUALCÓYOTL,AMBULATORIO,...,0,0,0,0,2020,4,4,16,17,108
2020-04-18,2020-06-01,098a35,2,4,BAJA CALIFORNIA,2,SINALOA,BAJA CALIFORNIA,TIJUANA,HOSPITALIZADO,...,0,0,1,1,2020,4,5,16,18,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-01,2020-06-01,11cd21,1,12,VERACRUZ DE IGNACIO DE LA LLAVE,2,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ,HOSPITALIZADO,...,0,0,0,0,2020,6,0,23,1,153
2020-05-30,2020-06-01,0527d5,1,12,CHIAPAS,2,CHIAPAS,CHIAPAS,SAN CRISTÓBAL DE LAS CASAS,HOSPITALIZADO,...,0,0,0,0,2020,5,5,22,30,151
2020-05-20,2020-06-01,101d66,2,11,CIUDAD DE MÉXICO,2,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,GUSTAVO A. MADERO,AMBULATORIO,...,0,0,0,0,2020,5,2,21,20,141


## actualizar_datos_salud

In [ ]:
# exports
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def actualizar_datos_salud(directorio_datos='./datos/secretaria_salud/', fecha_inicio='12-04-2020'):
    '''
        Descarga todos los archivos de datos hasta el día anterior a la fecha actual. Si los archivos
        ya existen en el directorio entonces no los descarga.
    '''
    url_salud_historicos = 'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/'
    hoy = date.today() - timedelta(days=1)
    
    fecha_inicio = datetime.strptime(fecha_inicio, "%d-%m-%Y")
    for fecha in daterange(fecha_inicio.date(), hoy):
        archivo_nombre = f'{fecha.strftime("%y%m%d")}COVID19MEXICO.csv.zip'

        archivo_ruta = os.path.join(directorio_datos, archivo_nombre)
        
        if os.path.exists(archivo_ruta):
            print(f'Ya existe {archivo_nombre}')
        else:
            print(f'Bajando datos {fecha.strftime("%d.%m.%Y")}')
            url_dia = f'{fecha.strftime("%m")}/datos_abiertos_covid19_{fecha.strftime("%d.%m.%Y")}.zip'
        
            url = url_salud_historicos + url_dia
            r = requests.get(url, allow_redirects=True)
            open(archivo_ruta, 'wb').write(r.content)


    
    
    

In [ ]:
actualizar_datos_salud(fecha_inicio='12-04-2020')

## agrupa_casos_municipios

In [ ]:
# export

def agrupa_casos_municipios(casos_df):
    '''
        Este método crea una tabla con todos los municipios y el número de casos registrados
            para cada tipo de paciente y para cada resultado.
    '''
    casos_df = casos_df.copy()

    # cuenta número de casos por resultado
    marco_2019 = gpd.read_file('datos/municipios/marco_2019.json')
    marco_2019.rename(columns={'CVE_ENT': 'CLAVE_ENTIDAD_RES',
                               'municipio_cvegeo': 'CLAVE_MUNICIPIO_RES',
                               'municipio_nombre': 'MUNICIPIO_RES'}, inplace=True)
    marco_2019.MUNICIPIO_RES = marco_2019.MUNICIPIO_RES.str.upper()
    
    # poblacion_2020 = pd.read_csv('datos/municipios/proyeccion_poblacion_2020.csv',
    #                         dtype={'cve_ent': object, 'cve_mun': object, 'pob2020': int})

    if 'geometry' in casos_df.columns:
        casos_df.drop(columns='geometry', inplaace=True)
        
    claves_municipio = marco_2019.CLAVE_MUNICIPIO_RES.unique()
    tipos_resultado = casos_df.RESULTADO.unique()
    
    # claves_entidades = casos_df[['CLAVE_ENTIDAD_RES', 'ENTIDAD_RES']].drop_duplicates()
    
    # primero armamos la tabla de municipios y todos los tipos de casos, para luego llenar los conteos de cada tipo
    # para cada municipio
    tipos_caso_municipio = set(itertools.product(claves_municipio, tipos_resultado))
    tipos_caso_municipio = pd.DataFrame(tipos_caso_municipio, columns={'CLAVE_MUNICIPIO_RES': str, 'RESULTADO': str})

    # agregamos nombres y claves de entidad
    # poblacion_2020.rename(columns={'cve_ent': 'CLAVE_ENTIDAD_RES', 'cve_mun': 'CLAVE_MUNICIPIO_RES'}, inplace=True)
    tipos_caso_municipio = marco_2019.merge(tipos_caso_municipio, on=['CLAVE_MUNICIPIO_RES'], how='left')
    tipos_caso_municipio = tipos_caso_municipio.merge(casos_df[['CLAVE_ENTIDAD_RES', 'ENTIDAD_RES']].drop_duplicates(),
                                                      on=['CLAVE_ENTIDAD_RES'], how='left')
    
    # ahora contamos que ha pasado en cada municipio 
    cols_localidad = ['ENTIDAD_RES', 'MUNICIPIO_RES', 'CLAVE_MUNICIPIO_RES']    
    covid_municipal = casos_df[cols_localidad + ['RESULTADO', 'DEFUNCION']].groupby(cols_localidad + ['RESULTADO'])

    # claves_entidades = casos_df[['CLAVE_ENTIDAD_RES', 'ENTIDAD_RES']].drop_duplicates()
    
    covid_municipal = covid_municipal.agg({'RESULTADO':'count', 'DEFUNCION':sum})    
    covid_municipal.rename(columns={'RESULTADO': 'conteo', 'DEFUNCION': 'defunciones'}, level=0, inplace=True)
    covid_municipal.reset_index(inplace=True)
    
    covid_municipal = covid_municipal.merge(marco_2019, on=['CLAVE_MUNICIPIO_RES', 'MUNICIPIO_RES'])
    
    covid_municipal = tipos_caso_municipio.merge(covid_municipal, on=list(tipos_caso_municipio.columns), how='left')
    covid_municipal[['conteo', 'defunciones']] = covid_municipal[['conteo', 'defunciones']].fillna(0)
    covid_municipal.drop(columns=['CVE_MUN'], inplace=True)
    return covid_municipal


In [ ]:
casos_municipales = agrupa_casos_municipios(casos_df)

In [ ]:
casos_municipales

,CLAVE_MUNICIPIO_RES,CLAVE_ENTIDAD_RES,MUNICIPIO_RES,geometry,RESULTADO,ENTIDAD_RES,conteo,defunciones
0,01001,01,AGUASCALIENTES,"POLYGON ((-102.09775 22.02325, -102.11598 22.0...",Resultado pendiente,AGUASCALIENTES,160.0,0.0
1,01001,01,AGUASCALIENTES,"POLYGON ((-102.09775 22.02325, -102.11598 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,652.0,30.0
2,01001,01,AGUASCALIENTES,"POLYGON ((-102.09775 22.02325, -102.11598 22.0...",No positivo SARS-CoV-2,AGUASCALIENTES,2756.0,11.0
3,01002,01,ASIENTOS,"POLYGON ((-102.04348 22.29392, -102.05861 22.3...",No positivo SARS-CoV-2,AGUASCALIENTES,32.0,0.0
4,01002,01,ASIENTOS,"POLYGON ((-102.04348 22.29392, -102.05861 22.3...",Positivo SARS-CoV-2,AGUASCALIENTES,8.0,0.0
...,...,...,...,...,...,...,...,...
7390,32057,32,TRANCOSO,"POLYGON ((-102.24610 22.73451, -102.24578 22.7...",No positivo SARS-CoV-2,ZACATECAS,10.0,2.0
7391,32057,32,TRANCOSO,"POLYGON ((-102.24610 22.73451, -102.24578 22.7...",Positivo SARS-CoV-2,ZACATECAS,3.0,0.0
7392,32058,32,SANTA MARÍA DE LA PAZ,"POLYGON ((-103.19774 21.58534, -103.23388 21.5...",No positivo SARS-CoV-2,ZACATECAS,1.0,1.0
7393,32058,32,SANTA MARÍA DE LA PAZ,"POLYGON ((-103.19774 21.58534, -103.23388 21.5...",Positivo SARS-CoV-2,ZACATECAS,0.0,0.0


In [ ]:
# exports

def leer_variables_municipales():
    variables_municipales = pd.read_csv('datos/municipios/indicadores.csv', index_col=False,
                                        dtype={'cvegeo': str, 'entidad_cv': str,
                                              'municipio_cvegeo': str, 'entidad_cvegeo': str},
                                        encoding='iso8859_2')
    arregla_cvegeo(variables_municipales, 'municipio_cvegeo')
    arregla_cvegeo(variables_municipales, extraer_ent=True)
    
    variables_municipales.drop(columns=['municipio_cvegeo', 'entidad_cv'], inplace=True)
    marco_2019 = gpd.read_file('datos/municipios/marco_2019.json')
    marco_2019.rename(columns={'municipio_cvegeo': 'cvegeo'}, inplace=True)
    # marco_2019 = marco_2019[['municipio_cvegeo', 'geometry', 'municipio_nombre']]
    variables_municipales = variables_municipales.merge(marco_2019, on='cvegeo')
    
    # variables_municipales.drop(columns='nom_mun', inplace=True)
    # variables_municipales.rename(columns={'municipio_nombre': 'nom_mun'}, inplace=True)
    return variables_municipales

In [ ]:
leer_variables_municipales()

,cvegeo,nom_ent,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,...,total_camas_publicos,hospitales_privados,total_de_camas_privados,total_camas,pob2020,ent_cvegeo,CVE_ENT,CVE_MUN,municipio_nombre,geometry
0,01001,Aguascalientes,877190,2.0570,3.4786,25.6873,14.1560,0.5478,0.5543,0.6627,...,1228,17,43,1271,961977,01,01,001,Aguascalientes,"POLYGON ((-102.09775 22.02325, -102.11598 22.0..."
1,01002,Aguascalientes,46464,4.4207,2.5959,41.8921,5.5355,1.6779,5.6457,1.2979,...,0,0,0,0,50864,01,01,002,Asientos,"POLYGON ((-102.04348 22.29392, -102.05861 22.3..."
2,01003,Aguascalientes,56048,4.7805,4.3906,49.1856,9.7524,1.0577,0.7756,0.9519,...,46,1,0,46,60760,01,01,003,Calvillo,"POLYGON ((-102.57625 21.96778, -102.59749 22.0..."
3,01004,Aguascalientes,15577,4.3233,2.6265,33.0597,5.1165,1.5629,2.8986,0.5968,...,0,0,0,0,16918,01,01,004,Cosío,"POLYGON ((-102.26605 22.40372, -102.28993 22.4..."
4,01005,Aguascalientes,120405,3.2445,4.1297,33.7271,13.8341,0.9201,1.0273,0.9443,...,297,0,0,297,130184,01,01,005,Jesús María,"POLYGON ((-102.31034 22.03716, -102.33259 22.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,32054,Zacatecas,19155,8.1332,3.6565,42.1699,8.8384,2.5438,8.9568,3.8478,...,0,0,0,0,20177,32,32,054,Villa Hidalgo,"POLYGON ((-101.65599 22.51381, -101.67475 22.5..."
2461,32055,Zacatecas,30240,4.7985,3.4198,53.7212,13.5516,0.9201,4.8484,1.7459,...,22,1,0,22,31804,32,32,055,Villanueva,"POLYGON ((-102.69428 22.62230, -102.70306 22.6..."
2462,32056,Zacatecas,146147,1.2432,2.1746,23.0304,14.0968,0.2485,1.2004,1.6826,...,337,5,0,337,155533,32,32,056,Zacatecas,"POLYGON ((-102.58542 22.81149, -102.59311 22.7..."
2463,32057,Zacatecas,19413,5.6122,2.5997,42.0488,9.0609,1.1950,6.9448,3.0891,...,12,0,0,12,20285,32,32,057,Trancoso,"POLYGON ((-102.24610 22.73451, -102.24578 22.7..."


In [ ]:
# exports
def unir_casos_estadisticas_municipales(casos_municipios_df, estats_municipios):
    cols_com = columnas_comunes(casos_municipios_df, estats_municipios)
    casos_municipios_df = casos_municipios_df.merge(estats_municipios,
                                      left_on=['CLAVE_MUNICIPIO_RES'] + cols_com,
                                      right_on=['cvegeo'] + cols_com, how='right')
    
    casos_municipios_df.drop(columns=['CVE_ENT', 'entidad_cvegeo', 'nom_mun', 'cvegeo',
                                      'nom_ent', 'municipio_nombre', 'id', 'ent_cvegeo', 'pob01'], inplace=True)
    
    return casos_municipios_df

In [ ]:
# exports

def tabla_covid_indicadores_municipales(fecha, solo_positivos=True):
    covid_df = carga_datos_covid19_MX(fecha)
    covid_mun_df = agrupa_casos_municipios(covid_df)
    mun_df = leer_variables_municipales()
    mun_df.drop(columns=['geometry'])
    covid_mun_df = unir_casos_estadisticas_municipales(covid_mun_df, mun_df)
    
    if solo_positivos:
        covid_mun_df = covid_mun_df[covid_mun_df.RESULTADO == 'Positivo SARS-CoV-2'] 
    return covid_mun_df
    

In [ ]:
tabla = tabla_covid_indicadores_municipales('200601')

In [ ]:
tabla.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2465 entries, 1 to 7393
Data columns (total 87 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   CLAVE_MUNICIPIO_RES        2465 non-null   object  
 1   CLAVE_ENTIDAD_RES          2465 non-null   object  
 2   MUNICIPIO_RES              2465 non-null   object  
 3   geometry                   2458 non-null   geometry
 4   RESULTADO                  2465 non-null   object  
 5   ENTIDAD_RES                2465 non-null   object  
 6   conteo                     2465 non-null   float64 
 7   defunciones                2465 non-null   float64 
 8   pt_2015                    2465 non-null   int64   
 9   an_2015                    2454 non-null   float64 
 10  ne614_015                  2454 non-null   float64 
 11  bi_2015                    2454 non-null   float64 
 12  sins_15                    2454 non-null   float64 
 13  vpt_2015                 

In [ ]:
tabla

,CLAVE_MUNICIPIO_RES,CLAVE_ENTIDAD_RES,MUNICIPIO_RES,geometry,RESULTADO,ENTIDAD_RES,conteo,defunciones,pt_2015,an_2015,...,hospitales_semar,hospitales_sedena,hospitales_pemex,total_hospitales_publicos,total_camas_publicos,hospitales_privados,total_de_camas_privados,total_camas,pob2020,CVE_MUN
1,01001,01,AGUASCALIENTES,"POLYGON ((-102.09775 22.02325, -102.11598 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,652.0,30.0,877190,2.0570,...,0,0,0,8,1228,17,43,1271,961977,001
4,01002,01,ASIENTOS,"POLYGON ((-102.04348 22.29392, -102.05861 22.3...",Positivo SARS-CoV-2,AGUASCALIENTES,8.0,0.0,46464,4.4207,...,0,0,0,0,0,0,0,0,50864,002
7,01003,01,CALVILLO,"POLYGON ((-102.57625 21.96778, -102.59749 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,11.0,0.0,56048,4.7805,...,0,0,0,1,46,1,0,46,60760,003
9,01004,01,COSÍO,"POLYGON ((-102.26605 22.40372, -102.28993 22.4...",Positivo SARS-CoV-2,AGUASCALIENTES,15.0,0.0,15577,4.3233,...,0,0,0,0,0,0,0,0,16918,004
14,01005,01,JESÚS MARÍA,"POLYGON ((-102.31034 22.03716, -102.33259 22.0...",Positivo SARS-CoV-2,AGUASCALIENTES,30.0,2.0,120405,3.2445,...,0,0,0,1,297,0,0,297,130184,005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,32054,32,VILLA HIDALGO,"POLYGON ((-101.65599 22.51381, -101.67475 22.5...",Positivo SARS-CoV-2,ZACATECAS,1.0,1.0,19155,8.1332,...,0,0,0,0,0,0,0,0,20177,054
7385,32055,32,VILLANUEVA,"POLYGON ((-102.69428 22.62230, -102.70306 22.6...",Positivo SARS-CoV-2,ZACATECAS,2.0,0.0,30240,4.7985,...,0,0,0,1,22,1,0,22,31804,055
7388,32056,32,ZACATECAS,"POLYGON ((-102.58542 22.81149, -102.59311 22.7...",Positivo SARS-CoV-2,ZACATECAS,60.0,4.0,146147,1.2432,...,0,0,0,3,337,5,0,337,155533,056
7391,32057,32,TRANCOSO,"POLYGON ((-102.24610 22.73451, -102.24578 22.7...",Positivo SARS-CoV-2,ZACATECAS,3.0,0.0,19413,5.6122,...,0,0,0,1,12,0,0,12,20285,057


## Obtener municipios urbanos

In [ ]:
#exports

def municipios_urbanos():
    archivo = 'datos/municipios/municipios_ciudades.csv'
    
    df = pd.read_csv(archivo, dtype=str)
    df.dropna(inplace=True)
    df.rename(columns={'municipio_cvegeo': "CLAVE_MUNICIPIO"}, inplace=True)
    return df

In [ ]:
municipios_urbanos()

In [ ]:
from nbdev.export import notebook2script
notebook2script()